# 00-parsing_antiSMASH



### Overview

Content: 
1. Read in antismash output
2. retreive Regulators seqs from differet families e.g. all LuxR tfs

In [1]:

import sys
import os
def list_of_objects_in_a_dir(dir_path:str): 
    """List all objects in a directory.
    
    Parameters
    ----------
    dir_path : str
        The path to the directory.
        
    Returns
    -------
    list
        A list of the objects in the directory.
    """
    # list to store files
    list_of_files = []

    # Iterate directory
    for path in os.listdir(dir_path):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path, path)):
            list_of_files.append(path)
            
    return list_of_files


In [2]:
from teemi.design.fetch_sequences import read_genbank_files
import pandas as pd

In [3]:
go4010 = read_genbank_files('../wet_lab_notebooks/data_for_drylab/Go40110_flye_polish_actinoannotPFAM_antismash7/Go40110_flye_polish_actinoannotPFAM.gbk')[0]

In [4]:
f'Lenght of the genome {len(go4010):,} bp'

'Lenght of the genome 9,637,008 bp'

In [5]:
go4010

SeqRecord(seq=Seq('CAGCTCCGGCACGGAAAGAGGCTGGTCTGGTTTTGTGGGGTGGGGGCTGGGGTT...AGG'), id='.', name='Exported', description='Streptomyces reticuli_0.928 strain strain', dbxrefs=[])

How many clusters

In [6]:
# How many genes do we have?
locus_tags = []
qualifiers = []
cds = []

for feature in go4010.features:
    if feature.type == 'CDS': 
        # getting the locus tag if we see cds
        locus_tags.append(feature.qualifiers['locus_tag'][0])
        qualifiers.append(feature.qualifiers)

        # All CDS features
        cds.append(feature)

        
print('Total coding sequences : ', len(locus_tags))    

Total coding sequences :  8154


In [7]:
cds[:10]

[SeqFeature(SimpleLocation(ExactPosition(202), ExactPosition(1000), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(3087), ExactPosition(3231), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(6166), ExactPosition(7426), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(9178), ExactPosition(9358), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(9504), ExactPosition(9690), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(14019), ExactPosition(14781), strand=1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(15280), ExactPosition(15472), strand=-1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(16206), ExactPosition(16326), strand=-1), type='CDS', qualifiers=...),
 SeqFeature(SimpleLocation(ExactPosition(16859), ExactPosition(17699), strand=-1), type='CDS', qualifiers=...),
 SeqFeatu

Lets retrieve the locus_tag for all transcriptional regualtors

In [8]:
regulators = []
regulator_descriptions = []
for qualifier in qualifiers: 
    if 'gene_kind' in qualifier:
        if 'regulatory' in qualifier['gene_functions'][0]:
            #print(qualifier['gene_functions'])
            if 'LuxR' in qualifier['gene_functions'][0] or 'SARP' in qualifier['gene_functions'][0]: 
                print(qualifier['gene_functions'][0])
                regulator_descriptions.append(qualifier['gene_functions'][0][30:])
                regulators.append(qualifier)
                
len(regulators)

regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 187.1; E-value: 4.1e-57)
regulatory (smcogs) SMCOG1149:LuxR family transcriptional regulator (Score: 972.3; E-value: 4.8e-294)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 166.8; E-value: 6.4e-51)
regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 314.4; E-value: 3.3e-95)
regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 259.2; E-value: 1.5e-78)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 66; E-value: 4.5e-20)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 143.4; E-value: 9.7e-44)
regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 266.8; E-value: 7.8e-81)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 151.1; E-value: 4.3e-46)
regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response re

18

In [9]:
regulator_descriptions

['LuxR family DNA-binding response regulator (Score: 187.1; E-value: 4.1e-57)',
 'LuxR family transcriptional regulator (Score: 972.3; E-value: 4.8e-294)',
 'LuxR family DNA-binding response regulator (Score: 166.8; E-value: 6.4e-51)',
 'transcriptional regulator, SARP family (Score: 314.4; E-value: 3.3e-95)',
 'transcriptional regulator, SARP family (Score: 259.2; E-value: 1.5e-78)',
 'LuxR family DNA-binding response regulator (Score: 66; E-value: 4.5e-20)',
 'LuxR family DNA-binding response regulator (Score: 143.4; E-value: 9.7e-44)',
 'transcriptional regulator, SARP family (Score: 266.8; E-value: 7.8e-81)',
 'LuxR family DNA-binding response regulator (Score: 151.1; E-value: 4.3e-46)',
 'LuxR family DNA-binding response regulator (Score: 175.3; E-value: 1.7e-53)',
 'transcriptional regulator, SARP family (Score: 252.9; E-value: 1.2e-76)',
 'LuxR family DNA-binding response regulator (Score: 84.7; E-value: 8.6e-26)',
 'transcriptional regulator, SARP family (Score: 511.2; E-value:

In [10]:
locus_tags_to_search_for = []
for qualifier in regulators: 
    locus_tags_to_search_for.append(qualifier['locus_tag'][0])

In [11]:
print(f'Total sequences within these two regulator families: {len(locus_tags_to_search_for)}')
locus_tags_to_search_for

Total sequences within these two regulator families: 18


['LLPMBPKK_00292',
 'LLPMBPKK_00328',
 'LLPMBPKK_00586',
 'LLPMBPKK_01488',
 'LLPMBPKK_02197',
 'LLPMBPKK_02209',
 'LLPMBPKK_02563',
 'LLPMBPKK_02633',
 'LLPMBPKK_02635',
 'LLPMBPKK_02637',
 'LLPMBPKK_02662',
 'LLPMBPKK_05992',
 'LLPMBPKK_06337',
 'LLPMBPKK_06422',
 'LLPMBPKK_06907',
 'LLPMBPKK_07531',
 'LLPMBPKK_07744',
 'LLPMBPKK_07949']

### Manual curation: 
-  LuxR 11
- manual SARP 8

### In which gene clusters are they found?

In [12]:

regions_names = list_of_objects_in_a_dir('../wet_lab_notebooks/data_for_drylab/Go40110_flye_polish_actinoannotPFAM_antismash7/')

genbank_regions = []
for gb in regions_names: 
    if gb.endswith('.gbk') and gb.startswith('..'): 
        genbank_regions.append(gb)

genbank_regions_sorted = sorted(genbank_regions)



In [13]:
# lets parse: 
all_genbanks = []
for region in genbank_regions_sorted: 
    all_genbanks.append( read_genbank_files(f'../wet_lab_notebooks/data_for_drylab/Go40110_flye_polish_actinoannotPFAM_antismash7/{region}')[0])

In [14]:
regions_for_each_locus_tag = []
for lc_tag in locus_tags_to_search_for: 
    for i in range(len(all_genbanks)): 
       for feature in all_genbanks[i].features:
            if feature.type == "CDS":
                if feature.qualifiers['locus_tag'][0] == lc_tag: 
                    # getting the locus tag if we see cds
                    locus_tag_and_region = {'locus_tag':lc_tag, 'region': f'region {i+1}'}
                    regions_for_each_locus_tag.append(locus_tag_and_region)
regions_df = pd.DataFrame(regions_for_each_locus_tag, columns =['locus_tag', 'region'])
regions_df['regulator_number'] = [f'Reg {i+1}' for i in range(len(regions_df))]

regions_df['description'] = regulator_descriptions
regions_df

,locus_tag,region,regulator_number,description
0,LLPMBPKK_00292,region 1,Reg 1,LuxR family DNA-binding response regulator (Sc...
1,LLPMBPKK_00328,region 1,Reg 2,LuxR family transcriptional regulator (Score: ...
2,LLPMBPKK_00586,region 4,Reg 3,LuxR family DNA-binding response regulator (Sc...
3,LLPMBPKK_01488,region 10,Reg 4,"transcriptional regulator, SARP family (Score:..."
4,LLPMBPKK_02197,region 12,Reg 5,"transcriptional regulator, SARP family (Score:..."
5,LLPMBPKK_02209,region 12,Reg 6,LuxR family DNA-binding response regulator (Sc...
6,LLPMBPKK_02563,region 13,Reg 7,LuxR family DNA-binding response regulator (Sc...
7,LLPMBPKK_02633,region 14,Reg 8,"transcriptional regulator, SARP family (Score:..."
8,LLPMBPKK_02635,region 14,Reg 9,LuxR family DNA-binding response regulator (Sc...
9,LLPMBPKK_02637,region 14,Reg 10,LuxR family DNA-binding response regulator (Sc...


Manually, we have inspected the clusters and we are interested in the products from  14, 28, 38

### All regulators

In [15]:
## If we just look at all regulators
transcriptional_regulators = []
for qualifier in qualifiers: 
    if 'product' in qualifier: 
        if 'transcriptional regulator' in  qualifier['product'][0]: 
            tf = [qualifier['locus_tag'],qualifier['product']]
            transcriptional_regulators.append(tf)
print(f'Total transcriptional regulators : {len(transcriptional_regulators)}')
transcriptional_regulators[:10]

Total transcriptional regulators : 531


[[['LLPMBPKK_00020'], ['XRE family transcriptional regulator']],
 [['LLPMBPKK_00064'], ['XRE family transcriptional regulator']],
 [['LLPMBPKK_00191'], ['LacI family transcriptional regulator']],
 [['LLPMBPKK_00217'], ['MerR family transcriptional regulator']],
 [['LLPMBPKK_00244'], ['HxlR family transcriptional regulator']],
 [['LLPMBPKK_00260'], ['LacI family transcriptional regulator']],
 [['LLPMBPKK_00283'], ['TetR family transcriptional regulator']],
 [['LLPMBPKK_00293'], ['MarR family transcriptional regulator']],
 [['LLPMBPKK_00328'], ['LuxR family transcriptional regulator']],
 [['LLPMBPKK_00353'], ['LysR family transcriptional regulator']]]

Lets get the sequences: 

In [16]:
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [17]:
my_regulator_sequences = []
for i in range(len(cds)): 
    if cds[i].qualifiers['locus_tag'][0] in locus_tags_to_search_for: 
        feature_sequence = SeqRecord(cds[i].extract(go4010.seq), id = cds[i].qualifiers['locus_tag'][0],name = cds[i].qualifiers['locus_tag'][0], description = cds[i].qualifiers['gene_functions'][0] )
        
        # annotation
        feature_location = FeatureLocation(start=0, end=len(feature_sequence))
        my_feature = SeqFeature(location=feature_location, type='gene', 
                                qualifiers={"name": cds[i].qualifiers['locus_tag'][0], 
                                            "product": cds[i].qualifiers['locus_tag'][0],
                                            "description": cds[i].qualifiers['gene_functions'][0]})
        feature_sequence.features.append(my_feature)
        
        # saving feature
        my_regulator_sequences.append(feature_sequence)

my_regulator_sequences

[SeqRecord(seq=Seq('GTGACCCAGCCCGCCGCCGACCGCCACATCCCCCAGGCCCCGAACCGGATGCCG...TGA'), id='LLPMBPKK_00292', name='LLPMBPKK_00292', description='regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 187.1; E-value: 4.1e-57)', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGATGAAGGGGATGCCGCCGGTGGCCCCGCCTGCTGGTTCCGGTTCGTACGGA...TGA'), id='LLPMBPKK_00328', name='LLPMBPKK_00328', description='regulatory (smcogs) SMCOG1149:LuxR family transcriptional regulator (Score: 972.3; E-value: 4.8e-294)', dbxrefs=[]),
 SeqRecord(seq=Seq('GTGACCGACACCCCCATCCGTCTCCTCCTGGCCGACGACCACCCCGTGGTCCGC...TGA'), id='LLPMBPKK_00586', name='LLPMBPKK_00586', description='regulatory (smcogs) SMCOG1016:LuxR family DNA-binding response regulator (Score: 166.8; E-value: 6.4e-51)', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGGCCGTGTGAGCGGGGACACATACGGCGGGGAACGGATCGACTTCCGCAAC...TGA'), id='LLPMBPKK_01488', name='LLPMBPKK_01488', description='regulatory (smcogs) SMCOG1041:transcriptional regulator, SARP family (Score: 

In [18]:
with open('../wet_lab_notebooks/data_for_drylab/GOE_regulators.gb', 'w') as outfile:
    for i in range(0, len(my_regulator_sequences)):
        my_regulator_sequences[i].annotations["molecule_type"] = "DNA"
        print(my_regulator_sequences[i].format("gb"), file = outfile)

In [19]:
f'Number of regulator sequences: {len(my_regulator_sequences)}'

'Number of regulator sequences: 18'